In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-experimental

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
import getpass
import os
import google.generativeai as genai
from dotenv import load_dotenv
load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)


In [4]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Artist  LIMIT 10;")

sqlite
['Album', 'Artist', 'Customer', 'Employee', 'Genre', 'Invoice', 'InvoiceLine', 'MediaType', 'Playlist', 'PlaylistTrack', 'Track']


"[(1, 'AC/DC'), (2, 'Accept'), (3, 'Aerosmith'), (4, 'Alanis Morissette'), (5, 'Alice In Chains'), (6, 'Antônio Carlos Jobim'), (7, 'Apocalyptica'), (8, 'Audioslave'), (9, 'BackBeat'), (10, 'Billy Cobham')]"

In [5]:
from langchain_community.agent_toolkits import create_sql_agent
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

# llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)
agent_executor = create_sql_agent(llm, db=db, agent_type="", verbose=True , handle_parsing_errors=True)

In [6]:
agent_executor.invoke(
    "How many albums does alis in chain have?"
)



> Entering new SQL Agent Executor chain...
I should first list all the tables in the database, then query the album table to get the count of albums by alis in chain.
Action: sql_db_list_tables
Action Input: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, TrackAction: sql_db_query
Action Input: SELECT COUNT(*) FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'Alice In Chains');[(1,)]Final Answer: 1

> Finished chain.


{'input': 'How many albums does alis in chain have?', 'output': '1'}

THIS IS CAHIN NOT AGENT

In [6]:
from langchain.chains import create_sql_query_chain
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-pro", temperature=0)
chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many employees are there"})
response

'```sql\nSELECT COUNT(*) AS "Number of Employees"\nFROM Employee;\n```'